# Preocupações com P-Hacking e Big Data

Esta seção abordará o p-hacking e outros problemas de big data. Até agora, aprendemos como utilizar modelos controlados para analisar a significância estatística e se podemos levar as descobertas a sério. No entanto, quando se introduz viés humano e grandes quantidades de dados, torna-se fácil "comprar" descobertas que parecem significativas e confiáveis, mas que, na verdade, são apenas mera coincidência. Abordaremos a falácia do atirador de elite do Texas, viés e fatores humanos no que diz respeito à pesquisa científica.

> "Quando você tortura os dados por tempo suficiente, eles confessam qualquer coisa." - Ronald H. Coase

## Falácia do atirador de elite do Texas

Vamos fazer um experimento bem simples. Digamos que você se aproxima de um celeiro e atira na parede sem um alvo específico, como simulado abaixo.

In [ ]:
from matplotlib import pyplot as plt
import random 

x, y = [random.randrange(0,5)], [random.randrange(0,5)]
plt.xlim(-.5, 5.5)
plt.ylim(-.5, 5.5)
plt.plot(x, y, marker="x", markersize=10, color='red')
plt.show()

Agora pergunte a si mesmo: qual é a probabilidade de acertar aquele ponto específico que você acabou de atirar? Bem... quando você considera a parede inteira e o número infinito de pontos para acertar nela, você perceberá que é incrivelmente improvável. Para reforçar meu ponto, vamos desenhar um alvo ao redor daquele buraco que acabamos de criar.

In [ ]:
plt.xlim(-.5, 5.5)
plt.ylim(-.5, 5.5)
plt.plot(x, y, marker="o", markersize=36, color='red')
plt.plot(x, y, marker="o", markersize=26, color='blue')
plt.plot(x, y, marker="o", markersize=16, color='yellow')
plt.plot(x, y, marker="x", markersize=10, color='red')
plt.show()


Agora vou fazer algo ridículo. Vou chamar meus amigos e mostrar minha incrível pontaria. "Olha o alvo que acabei de acertar!", digo. "É tão improvável que eu acertasse EXATAMENTE neste ponto, e mesmo assim acertei!" Pense por um momento qual é o problema e siga em frente.

O problema é que, embora acertar um alvo específico seja extremamente improvável, é fácil apontar um alvo *depois* de ele ter acontecido. Posso atirar às cegas em uma parede, desenhar um alvo ao redor do buraco e apontar o ponto improvável que acabei de acertar. *Eu nunca previ o alvo, eu o observei depois que ele aconteceu.* Isso é o que chamamos de **Falácia do Atirador de Elite do Texas** e acontece com muita frequência em pesquisas científicas.

Considere outro exemplo: a probabilidade de um jogador específico ganhar na loteria é extremamente improvável, mas alguém *vai* ganhar na loteria. Vamos simular isso abaixo, supondo que haja apenas um ganhador em um milhão de competidores.

In [ ]:
import random 

def loteria(num_de_participantes: int): 
    ganhador = random.randint(0,num_de_participantes)
    probabilidade = 1 / num_de_participantes
    
    print(f"JOGADOR {ganhador} GANHOU NA LOTERIA!")
    print(f"Ele tinha {round(probabilidade * 100, 4)}% de chance de ganhar.")
          

loteria(1_000_000)

Estamos surpresos com o fato de haver um vencedor? Não! Foi pura sorte. Se previmos o vencedor, isso seria impressionante e útil. Mas não há nada de extraordinário em uma pessoa específica vencer quando ninguém previu que ela venceria. E, no entanto, essa pessoa tinha uma chance extremamente improvável de vencer: 0,0001%.

**A probabilidade de um evento improvável é provável! Só não sabemos qual.**

Essa falácia também é fácil de ser feita com dados e análises. Quanto mais dados temos, mais alvos podemos encontrar que foram coincidências aleatórias. Os alvos podem ter um valor-p notavelmente baixo e, ainda assim, serem coincidentes e considerados significativos. Isso é o que chamamos de p-hacking.

## Mineração de Dados e o Paradoxo de Simpson

Pergunte a si mesmo: mais dados são uma coisa boa?

Claro, ter mais dados oferece mais oportunidades de coletar informações. Mas quando se trata de descobertas estatísticas, a situação se torna cada vez mais precária. Podemos cair na armadilha de encontrar cada vez mais alvos que, por acaso, foram atingidos, mas o fizeram por coincidência. Podemos argumentar que os valores de p para essas descobertas são baixos e significativos. No entanto, como dito anteriormente, a probabilidade de eventos improváveis ​​é provável, apenas não sabemos quais. Eles são fáceis de identificar em retrospectiva.

Vejamos um conjunto de dados que estuda possíveis variáveis ​​que contribuem para o latido de um cachorro.

In [ ]:
import pandas as pd 

df = pd.read_csv("https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/797f82f66f2283bcb75d5ff8275c39f45167c2b6/regression/dog_barking.csv")
df

A coluna da direita representa o número de latidos, ou a variável de saída em que estamos interessados. As outras variáveis ​​incluem idade, sexo, temperatura ambiente, nível de ruído, número de batidas na porta e o número de horas que o dono ficou em casa. Vamos construir uma matriz de correlação, que mede a força de uma correlação/relação entre cada variável. Um valor de correlação de 0 indica ausência de correlação, enquanto um valor próximo a -1 ou 1 indica uma forte correlação negativa ou positiva, respectivamente.

In [ ]:
df.corr(method='pearson')

Parece haver uma correlação positiva decente entre `DOOR_KNOCKS` e `NUMBER_OF_BARKS`. Isso faz sentido. Cães tendem a latir quando alguém bate na porta da frente. Mas as outras variáveis ​​têm correlações muito fracas.

Poderíamos encerrar nosso estudo aqui, e devemos! Mas digamos que o pesquisador esteja desesperado para encontrar algo "interessante" ou "inovador". Então, o pesquisador agora divide os dados em diferentes faixas etárias.

Ele observa os cães abaixo, com idades entre 4 e 7 anos. Estranhamente, a correlação desaparece entre `DOOR_KNOCKS` e `NUMBER_OF_BARKS` neste grupo, e agora é `-0,054233`. Isso é o que chamamos de **Paradoxo de Simpson**, em que um padrão é revertido após ser segmentado.

In [ ]:
filtered_df = df[(df["AGE"] <= 7) & (df["AGE"] >= 4)]
filtered_df.corr(method='pearson')

Em seguida, ele observa cães entre 8 e 11 anos. Curiosamente, parece que cães na faixa etária de 8 a 11 anos latem menos quando o nível de ruído é maior, já que a correlação entre as duas variáveis ​​é `-0,700051`.

In [ ]:
filtered_df = df[(df["AGE"] <= 11) & (df["AGE"] >= 8)]
filtered_df.corr(method='pearson')

Agora, o pesquisador começa a levantar hipóteses. Talvez cães na faixa etária de 4 a 7 anos percam o interesse em latir quando batem na porta depois de fazer isso quando filhotes, mas retornem quando não estiverem mais esgotados! Talvez cães na faixa de 8 a 11 anos tenham mais dificuldades auditivas e os níveis de ruído ambiente abafem quaisquer outros estímulos que os façam latir.

Viu como estamos enlouquecendo minerando os dados em busca de descobertas, e continuamos fragmentando e manipulando os dados tentando encontrar conclusões interessantes? Essa prática lhe parece familiar?

![charlie](https://i.giphy.com/media/l0IylOPCNkiqOgMyA/giphy.webp "charlie")

*Cortesia: 20th Century Fox* 

Surpresa! Esses dados são gerados aleatoriamente, mostrando como é fácil fazer descobertas espúrias até mesmo em dados aleatórios. Essa busca por padrões nos dados é conhecida como mineração de dados, e as correlações que encontramos ocorreram completamente por acaso. Para maior credibilidade, determinei uma correlação positiva entre batidas de porta e latidos de cachorro, mas todos os outros campos eram completamente aleatórios. Isso mostra que correlações podem ser encontradas até mesmo em dados gerados aleatoriamente e, portanto, podem não ter sentido. Correlações aleatórias podem existir no mundo real e é outro motivo pelo qual o mantra "correlação não significa causalidade" é tão importante.

Se você quiser ver o código exato que foi usado para criar este conjunto de dados, aqui está a declaração Pandas/NumPy abaixo.

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

n = 40
np.random.seed(7)
door_knocks =  np.random.normal(3, 1, n).astype(int)

df =  pd.DataFrame({"AGE" : np.random.randint(0,16,n),
                    "SEX" : np.random.randint(0,2, n),
                    "ROOM_TEMPERATURE" : np.random.normal(76, 4, n),
                    "NOISE_LEVEL": np.random.normal(30, 10, n),
                    "DOOR_KNOCKS" : door_knocks,
                    "HOURS_OWNER_HOME" : np.random.normal(7, 3, n),
                    "NUMBER_OF_BARKS" :  door_knocks * np.random.binomial(2,.85, n)
                   })

df

O problema com os exemplos anteriores é que praticamos mineração de dados em vez do método científico.

**Mineração de dados:** Coletar dados e, em seguida, formular hipóteses

**Método científico:** Formular hipóteses e, em seguida, reunir dados

O método científico exige uma hipótese estruturada, e os dados são deliberadamente coletados para comprovar e refutar essa hipótese, utilizando um grupo de teste e um grupo de controle. A mineração de dados é um processo aberto, no qual coletamos muitos dados e esperamos encontrar padrões ocultos e insights (não testados) que podem não ser óbvios ou mesmo fazer sentido! Se a mineração de dados for utilizada, o ideal é que ela seja complementada com novos dados e testes, mas muitos profissionais não conseguem fazer isso.

> **Aprendizado de Máquina é Mineração de Dados?**

> O aprendizado de máquina percorre um enorme espaço de hipóteses em busca de hipóteses que se correlacionem com os dados, portanto, é, de fato, mineração de dados. Pense no aprendizado de máquina como uma ferramenta automatizada de mineração de dados, que avalia inúmeras relações de variáveis ​​nos dados. Quando treinamos um classificador de imagens para reconhecer vacas, estamos minerando dados para encontrar pixels que formam formas que se correlacionam com o rótulo "vaca". Embora isso possa ser útil, não é incomum encontrar correlações sem sentido ou incorretas entre grupos de pixels. Por exemplo, um classificador de imagens pode reconhecer campos vazios como "vacas" porque foi treinado em campos com vacas, mas correlacionou-se com o campo e não com as vacas.
As vacas coincidiram com o campo e, portanto, resultaram em uma hipótese que se correlacionou com as variáveis ​​incorretas.

## P-Hacking

P-hacking é a seleção seletiva de modelos e dados que produz um resultado desejado em vez de um resultado realista. Mais especificamente, P-hacking é a mineração de dados para um valor-p inferior a 0,05. Práticas indisciplinadas e sob pressão costumam ser a causa. Simplesmente escolher um modelo porque "parece significativo" ou "resolve meu objetivo", em vez de desafiá-lo, é uma maneira sutil e fácil de fazer P-hacking.

Isso pode levar a uma inflação de falsos positivos, onde nosso modelo se torna muito otimista em relação a um resultado, mas apresenta um desempenho ruim na prática. P-hacking é supostamente responsável pela crise de replicabilidade1, e pode-se argumentar que é agravada pela disponibilidade de dados e aprendizado de máquina.

**Exemplos de P-Hacking:**
* Coletar apenas dados suficientes para obter o resultado desejado.

* Removendo dados inconvenientes como "outliers" ou "ruído"
* Buscando variáveis ​​que geram o resultado desejado
* Dividindo os dados em subgrupos e focando em um grupo
* Buscando parâmetros do modelo que geram o resultado correto
* Usando sementes aleatórias que produzem os resultados desejados

**Motivações para o P-Hacking**
* Pressão da pesquisa: "Sem papel, sem financiamento"
* Pressão do trabalho: "Nosso cliente quer ver um modelo que preveja uma economia de 10% nos custos de transporte"
* Pressão da startup: "Nossos investidores de capital de risco querem uma demonstração, então encontre um conjunto de dados que produza resultados favoráveis."

O P-Hacking é malicioso e enganoso? Geralmente não, muitas vezes é a natureza humana operando sob pressão e sobrevivência na carreira.

## Viés de dados

Como humanos, somos estranhamente programados para sermos tendenciosos. Tendemos a procurar padrões em vez de raciocinar sobre por que eles podem nos enganar. O viés de dados é inevitável em estatística e ciência de dados, portanto, você deve sempre procurá-lo ativamente. Se eu fizer uma pesquisa com estudantes universitários da minha universidade local e usar minhas descobertas para representar todas as universidades dos Estados Unidos, obviamente haverá viés. Estou favorecendo a representação dessa universidade em detrimento de outras universidades e fazendo generalizações com base em uma amostra limitada.

Existem muitos tipos de viés, mas um dos mais perniciosos em estatística é o viés de *autosseleção*. Isso ocorre quando certos tipos de sujeitos têm maior probabilidade de se incluir no experimento. Se você embarcar em um voo e perguntar aos passageiros qual é a companhia aérea favorita deles, não se surpreenda se for exatamente a mesma companhia aérea em que estão voando! Se você estiver realizando pesquisas por telefone no meio de um dia de semana, não se surpreenda se a maioria dos seus entrevistados for aposentada, pais que ficam em casa e indivíduos que não trabalham. Tudo isso pode distorcer suas descobertas, pois a população pretendida da qual você deseja amostrar não está se incluindo na amostra.

Uma variante do viés de autosseleção é o **viés de sobrevivência**, em que apenas a população "sobrevivente" é incluída na amostra e aqueles que "pereceram" nunca são contabilizados.

O viés de sobrevivência acontece sutilmente. Empresas e indivíduos "bem-sucedidos" costumam ser alvo de críticas, e há muitos livros que analisam suas qualidades, mas não levam em conta que essas qualidades também podem ser comuns a empresas e indivíduos fracassados ​​que fracassaram na obscuridade.

Em resumo, desconfie do viés e pergunte de onde vieram os dados e o que pode tê-los direcionado, intencional ou não. Fazer isso evitará enormes dores de cabeça e projetos que dão errado. O viés não desaparece com mais dados. Isso só cria mais oportunidades para que o preconceito se instale.

## Exercício

svg image

Um fornecedor entrou em contato com a sua empresa de aeronaves militares. Eles possuem um modelo estatístico de "IA" que utiliza dados coletados de aeronaves retornando de combate e prevê onde a blindagem leve precisa ser instalada. O modelo utiliza dados detalhados e técnicas estatísticas para identificar onde é provável encontrar buracos de bala e danos. Em seguida, recomenda a blindagem desses pontos críticos.

Que perguntas você tem para o fornecedor? O método deles é sólido? Por que sim ou por que não? Há algum fator faltando? Pense bem sobre isso antes de prosseguir para a resposta.

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

svg image

Esta é uma versão modernizada de um problema estatístico real da Segunda Guerra Mundial. (https://apps.dtic.mil/docs/citations/ADA091073)

O Centro de Análises Navais conduziu um estudo sobre a mitigação da perda de bombardeiros. Após analisar frotas de bombardeiros retornados de missões, eles concluíram que superfícies que estatisticamente apresentam os maiores danos devem ser priorizadas para maior blindagem. Mas um matemático húngaro chamado Abraham Wald apontou uma falha fatal nessa heurística.

**A falha: os dados capturaram apenas aeronaves sobreviventes e, portanto, a abordagem estava completamente errada.**

Este é um exemplo de viés de sobrevivência, onde, novamente, fazemos inferências errôneas sobre a população sobrevivente, enquanto a população falecida nunca é contabilizada. Embora muitos digam cinicamente que os dados estão incompletos, eles ainda fornecem uma pista valiosa para resolver nosso objetivo. A pergunta que deveríamos estar fazendo: por que a aeronave retornou em segurança, apesar dos danos observados?

Com sucesso, Abraham inverteu a teoria ao blindar as partes não danificadas da aeronave, inferindo que essas eram provavelmente as áreas críticas que faziam com que o avião caísse e nunca mais retornasse à base. Isso não só salvou aeronaves e vidas, como também foi um momento crucial para o esforço de guerra.